In [1]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

In [18]:
model = ChatOpenAI(temperature=0)
# Define your desired data structure.
from pydantic import BaseModel, Field
from enum import Enum

class EmailType(str, Enum):
    spam = "spam"
    reminder = "reminder"
    others = "others"

class EmailParser(BaseModel):
    email_sender: str = Field(
        description="The email address of the sender, indicating who sent the email."
    )
    email_receiver: str = Field(
        description="The email address of the recipient, indicating who is intended to receive the email."
    )
    email_body: str = Field(
        description="The main content of the email, which may include text, links, or other information. This field represents the message being conveyed from the sender to the receiver."
    )
    email_type: EmailType = Field(
        description="The classification of the email based on its content or purpose. It can be one of the following: "
                    "'spam' for unsolicited or irrelevant emails, "
                    "'reminder' for messages that serve as a reminder or notification, "
                    "'others' for messages that serve other than spam and reminder "
    )

model = ChatOpenAI(temperature=0)


In [32]:
email = """
Dear Jacob,  
  
Your opinion matters to us actually and we love you so much!!!!  
  
You are invited to participate in a short survey about local transportation
issues in the Bay Area.  Your opinions will be used for research purposes only
and your responses will remain completely confidential.  
  
Follow this link to the Survey:  
Take the Survey  
  
Or copy and paste the URL below into your internet browser:  
https://thepollingplace.qualtrics.com/jfe/form/SV_aibZ5geNkxuBng2?Q_DL=4BQDYnCmFixO9bW_aibZ5geNkxuBng2_CGC_wYZc0ZnW5tL60xa&Q_CHL=email  
  
Thank you for your participation!  
  
EMC Research is a national market and opinion research firm that conducts
research on a variety of topics. We are not trying to sell anything, and your
responses will remain completely anonymous. To learn more about EMC Research,
go to www.emcresearch.com.  
  
Follow the link to opt out of future emails:  
Click here to unsubscribe


-----------

Dear Jacob,  
  
Your opinion matters to us!  
  
You are invited to participate in a short survey about local transportation
issues in the Bay Area.  Your opinions will be used for research purposes only
and your responses will remain completely confidential.  
  
Follow this link to the Survey:  
Take the Survey  
  
Or copy and paste the URL below into your internet browser:  
https://thepollingplace.qualtrics.com/jfe/form/SV_aibZ5geNkxuBng2?Q_DL=4BQDYnCmFixO9bW_aibZ5geNkxuBng2_CGC_wYZc0ZnW5tL60xa&Q_CHL=email  
  
Thank you for your participation!  
  
EMC Research is a national market and opinion research firm that conducts
research on a variety of topics. We are not trying to sell anything, and your
responses will remain completely anonymous. To learn more about EMC Research,
go to www.emcresearch.com.  
  
Follow the link to opt out of future emails:  
Click here to unsubscribe


Bamboo Spa Grand Reopening Sale!

|  |  
---  
|

4 Options Starting From

$70.00  
  
  
---  
  

View Offer  
  
What We're Offering

**Join in our reopening celebration today by enjoying exclusive offers on
your preferred services!**

Unwind and rejuvenate yourself with our exclusive packages of **Foot
Reflexology** , **Tension Tamer** , and **CustomBlend Massage**.

Make sure to visit us for our inaugural=C2=A0 **Halloween celebration on
October 27th, from 3pm-6pm </strong> (admission is free), where you can
indulge in delightful treats and take advantage of amazing deals **RSVP**
**.** For further details aboutthis month's promotions, kindly refer to our
**=C2=A0** **Specials Page.

**Special Offers for the Grand Reopening!**

****__* 1 hr Foot Reflexology $70__

____* 30 min Tension Tamer Massage + 30 min Foot Reflexology$85__

____* 60 min Custom Blend Massage $105__

____* 90 min Custom Blend Massage $125__

**Foot Reflexology:=C2=A0** Revitalizing and refreshing the feet and legs is
accomplished by stimulating key pressure points either manually or using a
reflexology stick, through foot reflexology.

**Tension Tamer:=C2=A0** Ease tension in your neck or unwindwith a soothing
scalp massage. Upon request, enjoy essential oils or lotion; otherwise
receive a dry treatment.

**Custom Blend Massage:** =C2=A0Our expert therapists with awealth of
experience will skillfully combine various techniques to relievetension,
promote relaxation and provide relief for your muscles.

For questions or more information, please call 415-567-8812 or visit our=
website at http://www.bamboospasf.com.  
  
|  

Bamboo Spa SF

415-567-8812

www.bamboospasf.com

frontdesk@bamboospasf.com

|

|

2284 Union St  
San Francisco, CA 94123  
  
---  
  
|  Find more local experiences and promotions  
|

  

|  
  
---|---  
This email was created by Hownd on behalf of Bamboo Spa SF  
(Located at 2284 Union St, San Francisco, CA, 94123).  
  
  
Please review our Privacy Policy.  
Unsubscribe to stop receiving this email.  
Manage Privacy
"""

In [35]:
import requests
from typing import Optional, List, Any

# Method 1
class CustomLLM(LLM):
    n: int

    def __init__(self):
        self.api_url = "http://localhost:9850" 

    def make_request(self, prompt):
        response = requests.post(
            self.api_url,
            json={"prompt": prompt} 
        )
        
        if response.status_code == 200:
            return response.json().get("response", "") 
        else:
            raise Exception(f"Error: {response.status_code}, {response.text}")

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        return self.make_request(prompt)
    
# Method 2
from langchain_community.chat_models import ChatOpenAI

llm = ChatOpenAI(
        openai_api_base="http://<host-ip>:<port>/<v1>",
        openai_api_key="dummy_value",
        model_name="model_deployed")

In [36]:
parser = JsonOutputParser(pydantic_object=EmailParser)

prompt = PromptTemplate(
    template="Can you provide details or insights on the very first email found at the top of the document below? \nMake sure to extract the email in the following format.{format_instructions}.\nHere is the document:\n{input}",
    input_variables=["input"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | local_api_tool | parser

chain.invoke({"input": email})

TypeError: Context.run() argument after ** must be a mapping, not StringPromptValue

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"$defs": {"EmailType": {"enum": ["spam", "reminder", "others"], "title": "EmailType", "type": "string"}}, "properties": {"email_sender": {"description": "The email address of the sender, indicating who sent the email.", "title": "Email Sender", "type": "string"}, "email_receiver": {"description": "The email address of the recipient, indicating who is intended to receive the email.", "title": "Email Receiver", "type": "string"}, "email_body": {"description": "The main content of the email, which may include text, links, or other information. T